In [1]:
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [2]:
torch.manual_seed(42)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device} ")

Using device: cuda 


## ***Build `CNN` from skratch.***

In [4]:
data_path = "../09-Hyperparameter-Tuning-Bayesian Search/fashion-mnist_train.csv"

In [5]:
import pandas as pd
df = pd.read_csv(data_path)

In [6]:
df.sample(3)

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
56049,3,0,0,0,0,0,0,0,0,0,...,47,70,47,39,41,47,37,0,0,0
46818,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2599,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [7]:
X = df.iloc[:, 1: ].values
y = df.iloc[:, 0].values

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [9]:
## Scaling the features
X_train = X_train/255.0
X_test = X_test/255.0

In [10]:
class MyCustomDataset(Dataset):
    def __init__(self, features, lable):
        self.features = torch.tensor(features, dtype=torch.float32).reshape(-1, 1, 28, 28) # batch size, 2d imgage
        self.lable = torch.tensor(lable, dtype=torch.long)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        return self.features[index], self.lable[index]

In [11]:
train_dataset = MyCustomDataset(X_train, y_train)
test_dataset = MyCustomDataset(X_test, y_test)

In [12]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, pin_memory=True, num_workers=4)
train_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, pin_memory=True, num_workers=4)

In [ ]:
class MyCNN(nn.Module):
    def __init__(self, input_features):
        super().__init__()

        # convlutions layer
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=input_features,  out_channels=32, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.MaxUnpool2d(kernel_size=2)
        )

        # pooling layer
        self.classifier = nn.Sequential()


    def forward(self, x):
        out = self.features(x)
        return self.classifier(out)